In [1]:
import lxml
import re
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
from requests import get

In [2]:
url1 = "https://www.imdb.com/search/title?count=100&title_type=feature,tv_series&ref_=nv_wl_img_2"

In [3]:
class IMDB(object):
	"""docstring for IMDB"""
	def __init__(self, url):
		super(IMDB, self).__init__()
		page = get(url)

		self.soup = BeautifulSoup(page.content, 'lxml')

	def articleTitle(self):
		return self.soup.find("h1", class_="header").text.replace("\n","")

	def bodyContent(self):
		content = self.soup.find(id="main")
		return content.find_all("div", class_="lister-item mode-advanced")

	def movieData(self):
		movieFrame = self.bodyContent()
		movieTitle = []
		movieDate = []
		movieRunTime = []
		movieGenre = []
		movieRating = []
		movieScore = []
		movieDescription = []
		movieDirector = []
		movieStars = []
		movieVotes = []
		movieGross = []
		for movie in movieFrame:
			movieFirstLine = movie.find("h3", class_="lister-item-header")
			movieTitle.append(movieFirstLine.find("a").text)
			movieDate.append(re.sub(r"[()]","", movieFirstLine.find_all("span")[-1].text))
			try:
				movieRunTime.append(movie.find("span", class_="runtime").text[:-4])
			except:
				movieRunTime.append(np.nan)
			movieGenre.append(movie.find("span", class_="genre").text.rstrip().replace("\n","").split(","))
			try:
				movieRating.append(movie.find("strong").text)
			except:
				movieRating.append(np.nan)
			try:
				movieScore.append(movie.find("span", class_="metascore unfavorable").text.rstrip())
			except:
				movieScore.append(np.nan)
			movieDescription.append(movie.find_all("p", class_="text-muted")[-1].text.lstrip())
			movieCast = movie.find("p", class_="")

			try:
				casts = movieCast.text.replace("\n","").split('|')
				casts = [x.strip() for x in casts]
				casts = [casts[i].replace(j, "") for i,j in enumerate(["Director:", "Stars:"])]
				movieDirector.append(casts[0])
				movieStars.append([x.strip() for x in casts[1].split(",")])
			except:
				casts = movieCast.text.replace("\n","").strip()
				movieDirector.append(np.nan)
				movieStars.append([x.strip() for x in casts.split(",")])

			movieNumbers = movie.find_all("span", attrs={"name": "nv"})

			if len(movieNumbers) == 2:
				movieVotes.append(movieNumbers[0].text)
				movieGross.append(movieNumbers[1].text)
			elif len(movieNumbers) == 1:
				movieVotes.append(movieNumbers[0].text)
				movieGross.append(np.nan)
			else:
				movieVotes.append(np.nan)
				movieGross.append(np.nan)

		movieData = [movieTitle, movieDate, movieRunTime, movieGenre, movieRating, movieScore, movieDescription,
							movieDirector, movieStars, movieVotes, movieGross]
		return movieData

In [4]:
if __name__ == '__main__':
	site1 = IMDB(url1)
	print("Subject: ", site1.articleTitle())
	data = site1.movieData()
	for i in range(len(data)):
		print(data[i][:]) #Print the data

Subject:  Feature Film/TV Series(Sorted by Popularity Ascending) 
['Polar', 'Extremely Wicked, Shockingly Evil, and Vile', 'The Punisher', 'Glass', 'Sex Education', 'Game of Thrones', 'True Detective', 'Bohemian Rhapsody', 'Aquaman', 'Vikings', 'You', 'Green Book', 'The Favourite', 'A Star Is Born', 'Once Upon a Time in Hollywood', 'Fast & Furious Presents: Hobbs & Shaw', 'Serenity', 'Roma', 'Birds of Prey (And the Fantabulous Emancipation of One Harley Quinn)', 'The Orville', 'Titans', 'Velvet Buzzsaw', 'Alita: Battle Angel', 'The Walking Dead', 'Outlander', 'Vice', 'Kingdom', 'Mortal Engines', 'Widows', 'Avengers: Endgame', 'Grace and Frankie', 'IO', 'Joker', 'Spider-Man: Into the Spider-Verse', 'Black Mirror', "Grey's Anatomy", 'Shameless', 'Russian Doll', 'Supernatural', 'Suspiria', 'How to Train Your Dragon: The Hidden World', 'Bird Box', 'Star Trek: Discovery', 'American Crime Story', 'Brooklyn Nine-Nine', 'Doragon bôru chô: Burorî', 'BlacKkKlansman', 'Riverdale', 'Peaky Blinders